<a href="https://www.kaggle.com/code/pruthvikmachhi/food-classification-knn-logistic-svm-rf-adaboost?scriptVersionId=177581301" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
df=pd.read_csv("/kaggle/input/online-food-dataset/onlinefoods.csv")

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df['Gender']

In [ ]:
df['Marital Status'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()

In [ ]:
columns=df.columns

In [ ]:

df['Occupation']=label.fit_transform(df['Occupation'])
df

In [ ]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
df = pd.get_dummies(df, columns=categorical_cols)
df


In [ ]:
import seaborn as sns 
sns.boxplot(df)

In [ ]:
y=df['Occupation']

In [ ]:

x=df.drop(['Occupation'],axis=1)

In [ ]:
y

In [ ]:
x

In [ ]:
from sklearn.feature_selection import mutual_info_classif
mi_scores = mutual_info_classif(x, y)

sorted_indices = np.argsort(mi_scores)[::-1]

for i in sorted_indices:
  print(f"{df.columns[i]}: {mi_scores[i]}")

In [ ]:
import matplotlib.pyplot as plt
importance=mutual_info_classif(x,y)
feat_imp=pd.Series(importance,x.columns[:])
feat_imp.plot(kind="barh",color="teal")
plt.show

In [ ]:
# Assuming you have a DataFrame named df and the mutual information scores in a dictionary named mutual_info_scores
mutual_info_scores = {
    'Monthly Income_10001 to 25000': 0.49157250393602614,
    'Age': 0.30081796900927427,
    'Gender_Female': 0.2359393736070261,
    'longitude': 0.23263381892330148,
    'Marital Status_Single': 0.12940124824115307,
    'Marital Status_Married': 0.09819775136770126,
    'Gender_Male': 0.08929953574382954,
    'Educational Qualifications_Ph.D': 0.08436743736658947,
    'Monthly Income_No Income': 0.08324765262074618,
    'Output_No': 0.07494049111042234,
    'Family size': 0.07377298034500135,
    'Educational Qualifications_Post Graduate': 0.059667014269953134,
    'Pin code': 0.054613516484299796,
    'Educational Qualifications_Graduate': 0.05260383293333648,
    'Monthly Income_More than 50000': 0.04849268094730186,
    'Monthly Income_25001 to 50000': 0.04069527217380764,
    'latitude': 0.03643127922699274,
    'Monthly Income_Below Rs.10000': 0.013004385104133709,
    'Educational Qualifications_Uneducated': 0.0,
    'Marital Status_Prefer not to say': 0.0,
    'Educational Qualifications_School': 0.0,
    'Output_Yes': 0.0
}

mi_scores_df = pd.DataFrame(mutual_info_scores.items(), columns=['Column', 'MI Score'])

columns_to_drop = mi_scores_df[mi_scores_df['MI Score'] < 0.1]['Column'].tolist()
x.drop(columns=columns_to_drop, inplace=True)
x


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4,random_state=42)

### SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,train_test_split
s=SVC()
parameter={'C':[0.1,10,100,0.01],"kernel":['rbf','linear','poly'],'gamma':['scale','auto']}
g=GridSearchCV(s,parameter,cv=4)

In [ ]:
g.fit(x_train,y_train)

In [ ]:
g.best_params_

In [ ]:
y_pred=d=g.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k=KNeighborsClassifier(3)

In [ ]:
k.fit(x_train,y_train)

In [ ]:
y_pred2=d=k.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred2))

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_train,y_train)

In [ ]:
y_pred3=model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred3,y_test)

### Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import GridSearchCV

In [ ]:
dt=DecisionTreeClassifier()
param = {
    'max_depth': [None, 5, 10, 15],              # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],              # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],                # Minimum number of samples required to be at a leaf node
    'max_features': [None, 'sqrt', 'log2'],    # Number of features to consider when looking for the best split
    'criterion': ['gini', 'entropy']             # Function to measure the quality of a split
}
gt=GridSearchCV(dt,param,cv=5,scoring='accuracy')

In [ ]:
gt.fit(x_train,y_train)

In [ ]:
para=gt.best_params_
para

In [ ]:
dt1=DecisionTreeClassifier(**para)
dt1.fit(x_train,y_train)

In [ ]:
y_pred4=dt1.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred4)

## Random Forest 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
param_grid = {
    'n_estimators': [100, 200, 300],            # Number of trees in the forest
    'max_depth': [None, 5, 10, 15],              # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],             # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],               # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2'],    # Number of features to consider when looking for the best split
    'criterion': ['gini', 'entropy']             # Function to measure the quality of a split
}
g2=GridSearchCV(rf,param_grid,cv=2)

In [ ]:
# g2.fit(x_train,y_train)

In [ ]:
# para2=g2.best_params_
# para2

In [ ]:
# rf1=RandomForestClassifier(**para2)

In [ ]:
# rf1.fit(x_train,y_train)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],        
    'learning_rate': [0.1, 0.5, 1.0] }      

ada_classifier = AdaBoostClassifier()

grid_search = GridSearchCV(ada_classifier, param_grid, cv=5, scoring='accuracy')

grid_search.fit(x_train, y_train)  

best_params = grid_search.best_params_

best_ada_classifier = AdaBoostClassifier(**best_params)

best_ada_classifier.fit(x_train, y_train) 

In [ ]:
y_pred6=best_ada_classifier.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred6)